# 모델 셀렉션 모듈

model_selection :
학습 데이터와 테스트 데이터 세트를 분리하거나 교차 검증 분할 및 평가, 그리고 Estimator의 하이퍼파라미터 튜닝을 위한 다양한 함수와 클래스를 제공

- Train data, Test data 분리하지 않고 머신러닝 수행

In [1]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [2]:
# 테스트 데이터 세트를 이용하지 않고 학습 데이터 세트로만 학습하고 예측할 때의 문제
iris = load_iris()
dt_clf = DecisionTreeClassifier()
dt_clf.fit(iris.data, iris.target)

DecisionTreeClassifier()

In [3]:
# 학습된 데이터로 예측 수행
pred = dt_clf.predict(iris.data)
accuracy_score(iris.target, pred)

1.0

위의 예측결과가 100% 정확한 이유는 이미 학습한 데이터 세트 기반으로 예측했기 때문

실제 예측에서는 학습하지 않은 데이터를 기반으로 예측을 해야함

때문에 학습데이터 자체를 학습데이터와 테스트 데이터로 분할하여 모델이 얼마나 잘 예측하는지 평가해봄

- cross_validate 메소드

In [4]:
from sklearn.model_selection import cross_validate

In [5]:
dtc = DecisionTreeClassifier()
cross_validate(dtc, iris.data, iris.target)

{'fit_time': array([0., 0., 0., 0., 0.]),
 'score_time': array([0.        , 0.00100136, 0.00100446, 0.00099015, 0.0009973 ]),
 'test_score': array([0.96666667, 0.96666667, 0.9       , 0.93333333, 1.        ])}

In [6]:
dtc = DecisionTreeClassifier()
cross_validate(dtc, iris.data, iris.target, return_train_score=True)

{'fit_time': array([0.00099754, 0.00100112, 0.00100303, 0.00099969, 0.        ]),
 'score_time': array([0.        , 0.00100136, 0.00099421, 0.        , 0.00099659]),
 'test_score': array([0.96666667, 0.96666667, 0.9       , 1.        , 1.        ]),
 'train_score': array([1., 1., 1., 1., 1.])}

- Train/Test 데이터 셋을 분리

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    iris.data, iris.target, test_size=0.2, random_state=2021
)

In [8]:
import pandas as pd
pd.Series(y_test).value_counts()

0    14
1    10
2     6
dtype: int64

- Stratified 분리

In [9]:

X_train, X_test, y_train, y_test = train_test_split(
    iris.data, iris.target, test_size=0.2, random_state=2021,
    stratify=iris.target
)
pd.Series(y_test).value_counts()

0    10
1    10
2    10
dtype: int64

- cross_val_score() method

In [10]:
from sklearn.model_selection import cross_val_score

In [11]:
# 성능 지표는 정확도(accuracy), 교차검증 세트는 5개
cross_val_score(dtc, iris.data, iris.target, cv=5)

array([0.96666667, 0.96666667, 0.9       , 0.96666667, 1.        ])

In [12]:
import numpy as np 
scores = cross_val_score(dtc, iris.data, iris.target, cv=5)
np.mean(scores)

0.9600000000000002

## GridSearchCV
- 교차 검증과 최적 하이퍼 파라메터 튜닝을 한꺼번에 수행

In [13]:
dtc = DecisionTreeClassifier(random_state=2021)

In [14]:
# parameter 를 dictionary 형태로 설정
params = {
    'max_depth': [2, 3, 4, 5],
    'min_samples_split': [2, 3]
}

In [15]:
from sklearn.model_selection import GridSearchCV
grid_dtc = GridSearchCV(dtc, param_grid=params, cv=3)

In [16]:
grid_dtc.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=2021),
             param_grid={'max_depth': [2, 3, 4, 5],
                         'min_samples_split': [2, 3]})

In [17]:
grid_dtc.cv_results_

{'mean_fit_time': array([0.00133109, 0.00099786, 0.00066511, 0.00132879, 0.00066511,
        0.00066495, 0.00066868, 0.00099428]),
 'std_fit_time': array([4.69014170e-04, 2.97360213e-07, 4.70302724e-04, 4.71822490e-04,
        4.70302885e-04, 4.70190333e-04, 4.72849116e-04, 8.10600176e-04]),
 'mean_score_time': array([0.00066304, 0.        , 0.0003322 , 0.00033228, 0.00033204,
        0.00033251, 0.00066431, 0.0003322 ]),
 'std_score_time': array([0.00046885, 0.        , 0.0004698 , 0.00046991, 0.00046957,
        0.00047025, 0.00046974, 0.0004698 ]),
 'param_max_depth': masked_array(data=[2, 2, 3, 3, 4, 4, 5, 5],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_min_samples_split': masked_array(data=[2, 3, 2, 3, 2, 3, 2, 3],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'max_depth': 2, 'min_samples_split'

In [18]:
# 최적 파라미터
grid_dtc.best_params_

{'max_depth': 2, 'min_samples_split': 2}

In [19]:
# 최고 정확도
grid_dtc.best_score_

0.9583333333333334

In [20]:

# 최고 정확도를 가지는 최적 파라미터로 학습한 estimator
best_estimator = grid_dtc.best_estimator_
best_estimator.score(X_test, y_test)

0.9